In [1]:
!pip install tweepy

     |████████████████████████████████| 147 kB 3.8 MB/s eta 0:00:01
You should consider upgrading via the '/Users/christopherkindl/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import tweepy
import numpy as np
import pandas as pd

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
number_of_tweets = 200
tweets = []
likes = []
time = []

for i in tweepy.Cursor(api.user_timeline, id = 'username', tweet_mode = 'extended').\
    items(number_of_tweets):
    tweets.append(i.full_text)
    likes.append(i.favorite_count)
    time.append(i.created_at)

In [ ]:
df = pd.DataFrame({'tweets': tweets, 'likes': likes, 'time':time})